# Summary:

#### In this notebook, the trial runs from the preceding notebook ('01_So_What.ipynb') are systematically extended to try to find an optimal configuration for the two models. The model parameters to be fine tuned are: number of epochs, number of layers and the number of units.

#### The best performing model configuration will be used in the notebook '03_Blue_in_Green.ipynb' to analyze the impact of adding noise to the clean dataset.


# Table of contents
* [1. Load modules](#Part1_link)
* [2. Setup data](#Part2_link)
<br >&nbsp;&nbsp;&nbsp;[2.1  Generate data, separate testing and validation set and standardize testing data](#Part2.1_link)
* [3. Setup models and evaluate for various hyper-parameter choices](#Part3_link)
<br >&nbsp;&nbsp;&nbsp;[3.1 Compile and fit LSTM and RNN model](#Part3.1_link)
* [4. Visualize results](#Part4_link)

<a id='Part1_link'></a>
# 1. Load modules

In [1]:
import sys
sys.path.append("../src/")
import Kind_of_Blue  # own class with a collection of methods used in this analysis

import tensorflow as tf

import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

import numpy as np
import pandas as pd


<a id='Part2_link'></a>
# 2. Setup data

<a id='Part2.1_link'></a>
### 2.1 Generate data, separate testing and validation set and standardize testing data

The following steps are repeated from the previous notebook, '01_So_What.ipynb', and are grouped into one single step here for simplicity.

In [2]:
# set a range of dates on which the observations are made
idx = pd.date_range(end='7/1/2020', periods=5*364, freq='d')

# take a sine function as the observations
num_periods = 10  # number of sine periods
observations = [np.sin(2*np.pi*num_periods*x/len(idx)) for x in range(len(idx))]
print('number of observations in time series: {}'.format(len(observations)))

# initialize dataframe to store time series
df = pd.DataFrame(data=observations, columns=['observations'])
df.index = idx

# initialize object
mdq = Kind_of_Blue.Kind_of_Blue()

# load dataframe into object
mdq._selected_features = ['observations']
mdq.df = df

# train-validation split ratio as class attribute set to 70%
print('train split ratio = ', mdq.TRAIN_SPLIT_RATIO)

# initialize dataset from dataframe 
mdq.initialize_dataset()
print('loaded data set length: {}'.format(len(mdq._dataset)))

# standardize data
mdq.standardize_data()

# check that mean equals zero and the standard deviation is one
print('mean: {}, std: {}'.format(round(np.mean(mdq._dataset), 2), round(np.std(mdq._dataset), 2)))

# set number of time points for 1/ future forecasting points and 2/ the past, historical time points
future_target_size = int(365/52)
past_history_size = int(1*365)

# set batch size
batch_size = 32

# generate train and validation data
mdq.generate_train_and_val_data(future_target_size=future_target_size, past_history_size=past_history_size
                                , batch_size=batch_size)

print('number of training samples: {}'.format(mdq._num_samples))

number of observations in time series: 1820
train split ratio =  0.7
loaded data set length: 1820
mean: 0.0, std: 1.0
debug3: check what buffer_size actually does! and why is data shape always (..., ..., 1) <- 1???
training set shape: x:(909, 365, 1), y:(909, 7, 1)
validation set shape: x:(174, 365, 1), y:(174, 7, 1)
number of training samples: 909


<a id='Part3_link'></a>
# 3. Setup models and evaluate for various hyper-parameter choices

<a id='Part3.1_link'></a>
### 3.1 Compile and fit LSTM and RNN model

In [3]:
# generator for configurations to be iterated over

def config_generator():
    
    unit_choices = [2, 8, 16, 64, 128, 256, 512]  # number of units in each neural network layer
    layer_choices = [2]  # total number of layers
    epoch_choices = [10, 30, 50]  # number of epochs the model is trained on
    
    for units in unit_choices:
        for num_layers in layer_choices:
            for epochs in epoch_choices:
                yield units, num_layers, epochs


In [ ]:
# iterations over the model parameter configurations are done for both LSTM as well as RNN model
# model_types = ['LSTM', 'RNN']
model_types = ['RNN']

# set number of steps per epoch
num_samples = mdq._num_samples
steps_per_epoch = int(num_samples/future_target_size)
validation_steps = int(steps_per_epoch/2)

# initialize results dictionary
res = {'model_type': [], 'epochs': [], 'num_layers': [], 'units': [], 'val_mse': []
       , 'mse': [], 'total_training_time': []}

for units, num_layers, epochs in config_generator():
    for model_type in model_types:

        print('currently running {} model'.format(model_type))

        # compile model
        mdq.compile_model(units=units, num_layers=num_layers, model_type=model_type)

        # fit model
        mdq.fit_model(epochs=epochs, steps_per_epoch=steps_per_epoch
                      ,validation_steps=validation_steps, model_type=model_type)
        
        # get errors
        history = mdq._histories[model_type]
        val_mse = history.history['val_mse'][-1]
        mse = history.history['mse'][-1]
        
        # get total training time
        total_training_time = sum(mdq._time_callbacks[model_type].times)
        
        # append results to results dictionary
        res['model_type'].append(model_type)
        res['epochs'].append(epochs)
        res['num_layers'].append(num_layers)
        res['units'].append(units)
        res['val_mse'].append(val_mse)
        res['mse'].append(mse)
        res['total_training_time'].append(total_training_time)


currently running RNN model
Epoch 1/10
129/129 [==============================] - 7s 57ms/step - loss: 0.9911 - mse: 0.9911 - val_loss: 0.9756 - val_mse: 0.9756
Epoch 2/10
129/129 [==============================] - 6s 47ms/step - loss: 0.9062 - mse: 0.9062 - val_loss: 0.8620 - val_mse: 0.8620
Epoch 3/10
129/129 [==============================] - 6s 48ms/step - loss: 0.7902 - mse: 0.7902 - val_loss: 0.7122 - val_mse: 0.7122
Epoch 4/10
129/129 [==============================] - 6s 50ms/step - loss: 0.6992 - mse: 0.6992 - val_loss: 0.5802 - val_mse: 0.5802
Epoch 5/10
129/129 [==============================] - 6s 46ms/step - loss: 0.6052 - mse: 0.6052 - val_loss: 0.4696 - val_mse: 0.4696
Epoch 6/10
129/129 [==============================] - 7s 53ms/step - loss: 0.5205 - mse: 0.5205 - val_loss: 0.3790 - val_mse: 0.3790
Epoch 7/10
129/129 [==============================] - 6s 50ms/step - loss: 0.4746 - mse: 0.4746 - val_loss: 0.3118 - val_mse: 0.3118
Epoch 8/10
129/129 [=====================

Epoch 22/50
129/129 [==============================] - 6s 45ms/step - loss: 0.2425 - mse: 0.2425 - val_loss: 0.0860 - val_mse: 0.0860
Epoch 23/50
129/129 [==============================] - 6s 43ms/step - loss: 0.2473 - mse: 0.2473 - val_loss: 0.0912 - val_mse: 0.0912
Epoch 24/50
129/129 [==============================] - 6s 48ms/step - loss: 0.2442 - mse: 0.2442 - val_loss: 0.0865 - val_mse: 0.0865
Epoch 25/50
129/129 [==============================] - 6s 44ms/step - loss: 0.2587 - mse: 0.2587 - val_loss: 0.0850 - val_mse: 0.0850
Epoch 26/50
129/129 [==============================] - 6s 47ms/step - loss: 0.2451 - mse: 0.2451 - val_loss: 0.0888 - val_mse: 0.0888
Epoch 27/50
129/129 [==============================] - 7s 51ms/step - loss: 0.2290 - mse: 0.2290 - val_loss: 0.0848 - val_mse: 0.0848
Epoch 28/50
129/129 [==============================] - 6s 46ms/step - loss: 0.2483 - mse: 0.2483 - val_loss: 0.0838 - val_mse: 0.0838
Epoch 29/50
129/129 [==============================] - 7s 51ms

Epoch 23/30
129/129 [==============================] - 8s 61ms/step - loss: 0.0688 - mse: 0.0688 - val_loss: 0.0097 - val_mse: 0.0097
Epoch 24/30
129/129 [==============================] - 8s 65ms/step - loss: 0.0693 - mse: 0.0693 - val_loss: 0.0061 - val_mse: 0.0061
Epoch 25/30
129/129 [==============================] - 9s 67ms/step - loss: 0.0706 - mse: 0.0706 - val_loss: 0.0127 - val_mse: 0.0127
Epoch 26/30
129/129 [==============================] - 10s 81ms/step - loss: 0.0694 - mse: 0.0694 - val_loss: 0.0124 - val_mse: 0.0124
Epoch 27/30
129/129 [==============================] - 8s 63ms/step - loss: 0.0672 - mse: 0.0672 - val_loss: 0.0066 - val_mse: 0.0066
Epoch 28/30
129/129 [==============================] - 14s 108ms/step - loss: 0.0702 - mse: 0.0702 - val_loss: 0.0104 - val_mse: 0.0104
Epoch 29/30
129/129 [==============================] - 8s 59ms/step - loss: 0.0703 - mse: 0.0703 - val_loss: 0.0069 - val_mse: 0.0069
Epoch 30/30
129/129 [==============================] - 8s 6

<a id='Part4_link'></a>
# 4. Visualize results

In [ ]:
# transform dictionary to dataframe
df_res = pd.DataFrame(res)

# store dataframe as csv locally
# df_res.to_csv('../data/02_results_run4.csv')

In [ ]:
# visualize results; use bubble plots to indicate magnitude of mean-square error for specific configuration comparing
# RNN to LSTM results

x_label = 'epochs'
y_label = 'units'
z_label = 'mse'
# condition_label = 'num_layers'
# condition_vals = list(set(df_res[condition_label]))

# condition_LSTM = (df_res['model_type']=='LSTM')
# condition_RNN = (df_res['model_type']=='RNN')

for model_type in ['LSTM', 'RNN']:
    condition_1 = (df_res['model_type']== model_type)
    
    x = df_res[condition_1][x_label].values
    y = df_res[condition_1][y_label].values
    
    z = df_res[condition_1][z_label].values
    plt.scatter(x, y, s=z*10000, alpha=0.6, c="red", linewidth=0.0)
        
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title('mean-square training error: {} model'.format(model_type))
    plt.show()

In [ ]:
df_res[df_res['model_type']=='RNN']